In [4]:
pip install caer

In [10]:
import caer
import os
import numpy as np
import cv2 as cv
import gc

In [11]:
IMG_SIZE = (80,80)
channels = 1
char_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset'

In [13]:
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path, char)))
    
# Sort in descending order.
char_dict = caer.sort_dict(char_dict, descending=True)
print(char_dict)

In [14]:
characters = []
count = 0
for i in char_dict:
    characters.append(i[0])
    count += 1
    if count >= 10:
        break
print(characters)

In [16]:
# Training
train = caer.preprocess_from_dir(char_path, characters, channels = channels, IMG_SIZE=IMG_SIZE, isShuffle=True)

In [17]:
len(train)

In [18]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
plt.imshow(train[0][0], cmap='gray')
plt.show()

In [19]:
featureSet, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [20]:
from tensorflow.keras.utils import to_categorical
# Normalize features
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(characters))

In [21]:
x_train,x_val,y_train,y_val = caer.train_val_split(featureSet, labels, val_ratio=.2)

In [22]:
del train
del featureSet
del labels
gc.collect()

In [23]:
pip install canaro

In [24]:
import canaro

In [25]:
BATCH_SIZE = 32
EPOCHS = 10

In [28]:
# Image data generator
datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)

In [32]:
#Creatng Model
model = canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE, channels=channels,output_dim=len(characters), 
                                          loss='binary_crossentropy', decay=1e-6, learning_rate=0.001, 
                                          momentum=0.9, nesterov=True)

In [33]:
model.summary()

In [36]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [38]:
training = model.fit(train_gen,
                    steps_per_epoch=len(x_train)//BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_val,y_val),
                    validation_steps=len(y_val)//BATCH_SIZE,
                    callbacks=callbacks_list)

In [54]:
test_path = r'../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/bart_simpson_18.jpg'
img = cv.imread(test_path)
plt.imshow(img)
plt.show()

In [55]:
def prepare(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, IMG_SIZE)
    img = caer.reshape(img, IMG_SIZE, 1)
    return img

In [56]:
predictions = model.predict(prepare(img))

In [57]:
print(characters[np.argmax(predictions[0])])